In [1]:
# vim: 
# author:
# date:
# content:

import numpy as np
import pandas as pd

In [2]:
# getattr()用于返回一个对象属性值，eg.getattr(a, 'bar')获取a的‘bar’属性值
# isinstance() 函数来判断一个对象是否是一个已知的类型，类似type()

In [3]:
def split(adata, columns, axis='obs'):
    '''Split AnnData by metadata column
    
    Args:
        adata (AnnData): The object to split
        columns (str or list): column(s) of metadata table to split by
        axis (str): 'obs' (default) or 'var'
        
    Returns:
        dict of AnnData with keys equal to the unique values found in that
        columns of the appropriate metadata (adata.obs or adata.var)
    '''
    if axis not in ('obs','var'):
        raise ValueError('axis must be "obs" or "var"')
    
    # getattr()用于返回一个对象属性值，eg.getattr(a, 'bar')获取a的‘bar’属性值
    meta = getattr(adata, axis)
    
    # isinstance() 函数来判断一个对象是否是一个已知的类型，类似type()
    if isinstance(columns, str):
        column = columns
        if column not in meta.columns:
            raise ValueError('column not found')
        metac = meta[column]
    else:
        if len(columns) < 1:
            return adata
        elif len(columns) == 1:
            return split(adata, columns[0], axis=axis)
        
        # Merge with @ (a hack, but alright in most cases)
        metac = meta[columns[0]].astype(str)
        for i, col in enumerate(columns[1:]):
            metac = metac + '@' + meta[col].astype(str)
            
    # Unique values
    metau = metac.unique()
    
    # Construct dict
    d = {}
    for key in metau:
        idx = metac.index[metac == key]
        if axis == 'obs':
            asub = adata[idx]
        else:
            asub = adata[:, idx]
    
        if not isinstance(columns, str) and (len(columns) > 1):
            key = key.split('@')
            for i, col in enumerate(columns):
                key[i] = type(meta[col].iloc[0])(key[i])
            key = tuple(key)

        d[key] = asub
    
    return d    

In [4]:
def average_expression(adata, columns, axis='obs'):
    '''Average expression by metadata column
    
    Args:
        adata (AnnData): The project to split
        columns (str or list): column(s) of metadata table to split by
        axis (str): 'obs' (default) or 'var'
        
    Returns:
        pandas DataFrame with rows equal to the non-integrated axis names and columns
        equal to the keys of the AnnData split.
    '''
    
    adatad = split(adata, columns, axis=axis)
    
    if axis == 'var':
        iax = 1
        index = adata.obs_names
    else:
        iax = 0
        index = adata.var_names
    
    ave_expre = {}
    
    for key, adatai in adatad.items():
        av_ex = np.asarray(adatai.X .mean(axis=iax))[0] # number of all genes
        # av_ex = np.asarray(adatai.X.sum(axis=iax) / (adatai.X > 0).sum(axis=iax))[0] # number of only expressed genes
        ave_expre[key] = av_ex
    ave_expre = pd.DataFrame(ave_expre, index=index)
    
    return ave_expre

In [5]:
# average_expression
import os 
import sys

import anndata
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

# sys.path.append('/home/yike/phd/function/')

if __name__ == '__main__':
    
    print('Load interaction')
    fn_int = '/home/yike/phd/dengue/data/interaction_unpacked_mouse.tsv'
    interactions = pd.read_csv(fn_int, sep='\t')[['gene_name_a', 'gene_name_b']]
    ga, gb = interactions['gene_name_a'], interactions['gene_name_b']
    
    if True:
        print('Load high-quality cells only')
        fn_h5ad = '/home/yike/phd/dengue/data/mergedata_20200930_high_quality.h5ad'
        adata = anndata.read_h5ad(fn_h5ad)
        adata.obs['dataset'] = adata.obs['platform'].replace({
            '10X': 'children',
            'plate': 'adult'   
        })
        sc.pp.normalize_total(adata, target_sum=1e6)
        sc.pp.log1p(adata)
        
    print('Restrict to interaction genes')
    genes = np.unique(interactions)
    adatag = adata[:, genes]
    
    print('Split by cell type, adult and children, and condition')
    # from anndata_utils.partition import expressing_fractions
    obs = adatag.obs
    adatag.obs['split_col'] = obs['dataset'] + '+' + obs['Condition'].astype(str) + '+' + obs['cell_type'].astype(str)
    
    split_cols = ['dataset', 'Condition', 'cell_type']
    
    # from function import average_expression
    ave_expre = average_expression(adatag, split_cols)
    
    from collections import defaultdict #自动产生任何新list或dict
    th = 0
    cell_types = list(obs['cell_type'].cat.categories)
    res = {}
    for col in ave_expre.columns:
        datas, cond, cell_type1 = col
        for cell_type2 in cell_types:
            col2 = (datas, cond, cell_type2)
            res[(datas, cond, cell_type1, cell_type2)] = []
            av_exa = ave_expre.loc[ga, col].values
            av_exb = ave_expre.loc[gb, col2].values
            ind = (av_exa > th) & (av_exb > th)
            ind = ind.nonzero()[0] # 用于得到数组array中非零元素的位置（数组索引）的函数
            
            for i in ind:
                resi = {
                    'dataset': datas,
                    'Condition': cond,
                    'cell_type1': cell_type1,
                    'cell_type2': cell_type2,
                    'gene_name_a': interactions.iloc[i]['gene_name_a'],
                    'gene_name_b': interactions.iloc[i]['gene_name_b'],
                    'av_exc1': av_exa[i],
                    'av_exc2': av_exb[i],
                }
                res[(datas, cond, cell_type1, cell_type2)].append(resi)
            res[(datas, cond, cell_type1, cell_type2)] = pd.DataFrame(res[(datas, cond, cell_type1, cell_type2)])     
    
    for value in res.values():
        value['av_ex_sum'] = value['av_exc1'] + value['av_exc2']
    
    # 把T_cell, B_cell 和B_cell和T_cell的数据合并在一起
    merge_res_list = []
    cell_types=['B_cells','NK_cells' , 'T_cells', 'Monocytes', 'Plasmablasts', 'pDCs', 'cDCs']
    for datas in ['children', 'adult']:
        for cond in ['S_dengue', 'dengue', 'Healthy', 'DWS']:
            for i, cell_type1 in enumerate(cell_types):
                for j, cell_type2 in enumerate(cell_types[: i+1]):
                    merge_res_list.append((datas, cond, cell_type1, cell_type2))
 
    res2={}
    for key, value in res.items():
        (datas, cond, cell_type1, cell_type2) = key
        key2 = (datas, cond, cell_type2, cell_type1)
        if key in merge_res_list:
            res[key2] = res[key2] [['dataset', 'Condition',
                              'cell_type2','cell_type1', 
                               'gene_name_b', 'gene_name_a',
                               'av_exc2','av_exc1', 'av_ex_sum']]
            res[key2].columns=['dataset', 'Condition',
                               'cell_type1','cell_type2', 
                               'gene_name_a', 'gene_name_b',
                               'av_exc1','av_exc2', 'av_ex_sum']
            res2[key]=pd.merge(res[key], res[key2],on=['dataset', 'Condition',
                               'cell_type1','cell_type2', 
                               'gene_name_a', 'gene_name_b',
                               'av_exc1','av_exc2', 'av_ex_sum'],
                              how='outer')
            res2[key] = res2[key].sort_values('av_ex_sum',ascending=False)

    for key in res2.keys(): 
        (datas, cond, cell_type1, cell_type2) = key
        res2[key].to_excel('/home/yike/phd/dengue/data/excels/average_expressions/all_genes/' 
                         + datas + '_' + cond + '_' + cell_type1 + '_' + cell_type2 + '.xls', index=False)

Load interaction
Load high-quality cells only


/usr/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Trying to set attribute `.obs` of view, copying.


Restrict to interaction genes
Split by cell type, adult and children, and condition


In [6]:
# (adatag.X > 0).mean(axis=0)
# (adatag.X > 0).sum(axis=0) # the number of expressed genes
# adatag.X.sum(axis=0) 
# adatag.X.mean(axis=0) 
# adatag.X.sum(axis=0))/((adatag.X > 0).sum(axis=0)